# Sudoku

[Sudoku](https://en.wikipedia.org/wiki/Sudoku) is a kind of puzzle game played on a square grid of 9 x 9 cells. Every cell must have a number from 1 to 9, but there are constraints on where the numbers can go:

1. A number cannot be repeated in the same row
2. or repeated in the same column
3. or repeated in the same "box", which is a region of 3 x 3 cells.

You can play Sudoku online at [Sudoku.com](https://sudoku.com/), or [Web Sudoku](https://www.websudoku.com/).

When Sudoku first became popular I tried to write a program to solve any puzzle. It was...not entirely successful. When I decided I wanted to learn how to use Jypter Notebooks better I thought a puzle solver might be fun to try again. So this exercise is as much about learning Jupyter, Pandas, Matplotlib and Python, as it is about Sudoku.

Also I'm practicing writing, even though no-one is going to read this.

## Definitions

Terminology varies a little between Sudoku sites, so here are the terms I settled on:

* A *region* is a *row*, *column*, or *box*.
* *Row* and *column* are fairly obvious. I use the term *box* to describe the 3x3 grids.
* A *cell* is the single element that contains a *value*.

## Contents <a class="anchor" id="contents"></a>

### [Overview of Solution Algorithms](#overview) (this notebook)

Introduction to the classes used to model Sudoku puzzles and their solving algorithms.

### [Benchmarking Sudoku Solver Performance](Sudoku/Performance.ipynb)

Testing the different algorithms against a range of [test files](data/sudoku_9x9/README.md), and some charting of the results.

### [Checking for Cheating](Sudoku/Cheating.ipynb)

Can I write a "cheating" solver that out performs all the "real" solvers?

### [Constraint Propogation Variability](Sudoku/Constraint%20Propogation%20Variability.ipynb)

I noticed that sometimes constraint propogation performance varied considerably *on the same test case*. I looked into it and found out why.

### [Solving Larger Sudoku Puzzles](Sudoku/Larger%20Puzzles.ipynb)

Testing algorithms on puzzles larger than the standard 9 x 9 grids (up to 25 x 25 cells).


---

# Overview of Solution Algorithms <a class="anchor" id="overview"></a>



## Working with Puzzles

A Sudoku puzzle is a kind of "[latin square](https://en.wikipedia.org/wiki/Latin_square)", which is an N x N array of N values, where no value is repeated in a row or column. Sudoku adds a variation for boxes which impose an additional constraint. There are N boxes (usually =9), and they are also a 2D array of N values. Boxes do not overlap so their dimenisons are $\sqrt{N}x\sqrt{N}$. On puzzle grids the boxes are usually outlined with a heavier line.

There are a few other puzzle types which build their own additional rules on top of latin squares, so I've constructed a class [LatinSquare](puzzle/latinsquare.py) which has the basic rules in common to all latin squares, and then a class [SudokuPuzzle](puzzle/sudoku.py) which adds the additional Sudoku rules.


In [1]:
import puzzle.sudoku as su

I also found myself repeating a few short functions for displaying results in Jupyter. Although most of them are short I felt things "read better" if they're [tucked away into another module](puzzle/jupyter_helpers.py). Not 100% sure about doing it this way, particularly the `import *` part (which you wouldn't do in a normal Python program), so will keep an eye out for how others handle repeated "boilerplate" code like this.

The [jupyer_helpers](puzzle/jupyter_helpers.py) module contains a constant with some CSS formatting to make Sudoku puzzles *look* like puzzles. My CSS is *terrible* but I could get enough working to at least format puzzles as tables without clobbering Pandas' relatively nice formatting when displaying DataFrames.

In [2]:
from puzzle.jupyter_helpers import *
display(HTML(SUDOKU_CSS))

Final note on packages used in these notebooks: `copy` is for making copies of puzzle arrays.

In [3]:
import copy

## How the Puzzle Classes Work

I've chosen to represent puzzles in memory as a 2D array of integers. In Python terms, this is a list of lists of ints. Some other options that I came across later was to represent a puzzle simply as a string:

In [4]:
example = su.SAMPLE_PUZZLES[0]['puzzle']
example

'89.4...5614.35..9.......8..9.....2...8.965.4...1.....5..8.......3..21.7842...6.13'

This format is really useful for storing test data, so I have a function for converting from the string to the array version, `su.from_string`:

In [5]:
su.from_string(example)

[[8, 9, None, 4, None, None, None, 5, 6],
 [1, 4, None, 3, 5, None, None, 9, None],
 [None, None, None, None, None, None, 8, None, None],
 [9, None, None, None, None, None, 2, None, None],
 [None, 8, None, 9, 6, 5, None, 4, None],
 [None, None, 1, None, None, None, None, None, 5],
 [None, None, 8, None, None, None, None, None, None],
 [None, 3, None, None, 2, 1, None, 7, 8],
 [4, 2, None, None, None, 6, None, 1, 3]]

As you can see, I'm using `None` to represent empty cells. I could have also used `0` - as long as it evaluates to `False`. I got the idea that using `None` would be more "Pythonic" but there's every chance this is nonsense. Using `None` _does_ make looking at puzzles in this way really hard to read in the notebook, so the `SudokuPuzzle` class has a method `as_html`. We can initialise a puzzle from the string and then print it like so:

In [5]:
puzzle = su.SudokuPuzzle(starting_grid=su.from_string(example))
display(HTML(puzzle.as_html()))

8,9,,4,,,,5,6
1,4,,3,5,,,9,
,,,,,,8,,
9,,,,,,2,,
,8,,9,6,5,,4,
,,1,,,,,,5
,,8,,,,,,
,3,,,2,1,,7,8
4,2,,,,6,,1,3


I do this often enough that there's a function `print_puzzle` defined in [jupyter_helpers](puzzle/jupyter_helpers.py).

The `SudokuPuzzle` class is responsible for enforcing the rules, and will raise an exception if an attempt is made to write a value that violates one of the constraints. Cell positions are referred to as `x, y` and start from `0`. The main methods we'll use are:

* `init_puzzle`: Initializes the puzzle and resets internal state.
* `get(x, y)` / `set(x, y, value)`: Reads or writes a cell's value. The value `None` is returned for an empty cell. The `set` method will raise an exception if the value you're attempting to write is not allowed based on the current puzzle state.
* `clear(x, y)`: Clears a value from a cell and updates the constraints (because that value is now allowed again in that region).
* `get_allowed_values(x, y)`: Returns a `set` containing all the values that *may* be written to the cell at `x, y`, because that value does not already exist in that region.
* `is_valid` / `is_solved`: Checks the puzzle's integrity; or whether or not it has been solved yet.

## How the Solver Classes Work

I wanted to try various solving algorithms, including the possibility of algorithms that "cheat". Therefore I have a separate set of classes for "solvers", which all have a `solve` method that takes an instance of a `PuzzleClass` and updates the cells in order to solve it. 

The solvers are all defined in the module [puzzle.sudoku](puzzle/sudoku.py), and the list can be found at runtime in the `su.SOLVERS` dictionary:

In [7]:
su.SOLVERS

{'backtracking': puzzle.sudoku.BacktrackingSolver,
 'constraintpropogation': puzzle.sudoku.ConstraintPropogationSolver,
 'deductive': puzzle.sudoku.DeductiveSolver,
 'sat': puzzle.sudoku.SATSolver}

A helper class, [SudokuSolver](puzzle/sudoku.py) will accept a key value from that dictionary and initialise a new instance of the appropriate solver class, which makes it easy to "loop through" all the solvers when we want to test performance of different algorithms.

There's really only one method used in this class, `solve`. It takes a `SudokuPuzzle` instance and updates it, returning `True` if it is claiming the puzzle has been solved (the puzzle itself can verify its status with `is_solved`). Most of the classes are able to solve *any* Sudoku puzzle, *if given enough time to do so*. However, there is one class (`DeductiveSolver`) which can't solve "hard" puzzles on its own.

The simplest use of the classes is shown below. The very first puzzle in `SAMPLE_PUZZLES` is pretty easy and can be reliably solved with any of the solvers.

In [6]:
def solve_side_by_side(example, solver_method, **args):
    puzzle = su.SudokuPuzzle(starting_grid=su.from_string(example['puzzle']))
    original = copy.deepcopy(puzzle)
    solver_method(puzzle)
    display(HTML(f"<p><b>{example['label']} (difficulty level {example['level']})</b></p>"))
    print_2_puzzles(original, puzzle, **args)
    return puzzle

In [7]:
example = su.SAMPLE_PUZZLES[0]
solver = su.SudokuSolver()
puzzle = solve_side_by_side(example, solver.solve)

8,9,,4,,,,5,6
1,4,,3,5,,,9,
,,,,,,8,,
9,,,,,,2,,
,8,,9,6,5,,4,
,,1,,,,,,5
,,8,,,,,,
,3,,,2,1,,7,8
4,2,,,,6,,1,3
8,9,3,4,7,2,1,5,6
1,4,6,3,5,8,7,9,2


---
# Sudoku Solution Algorithms

These are the algorithms implemented in the `*Solver` classes:


In [8]:
display(HTML('<p><ol><li>' + '</li><li>'.join([x for x in su.SOLVERS.keys()]) + '</li></ol></p>'))

We have some test puzzles in the `sudoku` module. Each puzzle is marked with a "difficulty level", which is based on the number of starting clues. For illustrating the algorithms below, we'll start with the easiest puzzle, `#0`. By the way, for debugging purposes you can `print` a puzzle and it will show the "string" representation.

In [10]:
example = su.SAMPLE_PUZZLES[0]
puzzle = su.SudokuPuzzle(starting_grid=su.from_string(example['puzzle']))
original = copy.deepcopy(puzzle)
puzzle

SudokuPuzzle(9, '89.4...5614.35..9.......8..9.....2...8.965.4...1.....5..8.......3..21.7842...6.13')

## Deductive Logic

This was where I started when I first wanted to try to write a Sudoku solver. There are some [common Sudoku strategies](https://www.sudokudragon.com/sudokustrategy.htm), meant for humans and not too hard to code. Their primary advantage is that they are *fast*, particularly compared to the "backtracking" algorithms we'll look at later. However they have two disadvantages:

1. The simpler algorithms are not always able to solve all puzzles. The more difficult ones aren't usually solved this way, and we have to fall back to one of the other methods below.
2. The more complex algorithms *may* be able to solve all puzzles (this is a controversial point in the Sudoku world), but they're difficult to code and test.

The three coded here are all relatively simple:

- Single Possibilities
- Only Squares
- Two out of Three Rule

These methods are all coded in the class `DeductiveSolver`. Because these methods can't solve all puzzles, this particular class has a fall back option (back tracking). We'll get to that later, so for now we'll tell the class to *not* use the fall back method if it gets stuck.

### Single Possibilities

Single possibilities are those cells for which there is only one possible value, all others already exist in that row, column or box. This is sometimes enough to solve very easy Sudoku puzzles. For example, here's a smaller Sudoku grid (4x4) with almost all values filled in.

In [16]:
little_puzzle = su.SudokuPuzzle(4, '12343.1221434321')
print_puzzle(little_puzzle)
little_puzzle.get_allowed_values(1, 1)

1,2,3,4
3,,1,2
2,1,4,3
4,3,2,1


{4}

The value for cell `1, 1` is easily seen -- `4` is the only possible value that can go here.

The full implementation of this method can be found in [su.DeductiveSolver.solve_single_possibilities](puzzle/sudoku.py). The algorithm basically iterates over all cells, and calculates the possible values remaining for a cell based on the values set in that cells shared regions. If it finds a cell with only 1 possible value, then we write that in to the cell and continue.

In [12]:
solver = su.DeductiveSolver(use_backtracking=False)
puzzle = solve_side_by_side(example, solver.solve_single_possibilities)

8,9,,4,,,,5,6
1,4,,3,5,,,9,
,,,,,,8,,
9,,,,,,2,,
,8,,9,6,5,,4,
,,1,,,,,,5
,,8,,,,,,
,3,,,2,1,,7,8
4,2,,,,6,,1,3
8,9,3,4,7,2,1,5,6
1,4,6,3,5,8,7,9,2


### Only Squares

The [solve_only_squares](puzzle/sudoku.py) algorithm is similar, but instead of examing each *cell* and asking "what values are allowed here?" it iterates over each *region* and asks "where can each value possibly go?". For example, it will iterate over each *row*, look at the values that are *missing* for that row, and for each of those values ask "where can I possibly put this value?" If there's only one cell where that value could possibly go, then we write the value into that cell and continue.

Often a cell that can be solved this way can also be solved with "single possibilities" but since that's not always the case we try both approaches in the `DeductiveSolver`.

Again, this method can solve some simple puzzles but it *will* fail on more difficult examples.

In [13]:
puzzle = solve_side_by_side(example, solver.solve_only_squares)

8,9,,4,,,,5,6
1,4,,3,5,,,9,
,,,,,,8,,
9,,,,,,2,,
,8,,9,6,5,,4,
,,1,,,,,,5
,,8,,,,,,
,3,,,2,1,,7,8
4,2,,,,6,,1,3
8,9,3,4,7,2,1,5,6
1,4,6,3,5,8,7,9,2


### Two out of Three

This idea comes from [Sudoku Dragon's Two out of Three Rule](https://www.sudokudragon.com/forum/twothreestrategy.htm). The code is more complicated than any of the other methods which makes its performance dissapointing: On its own, it is not able to solve any puzzles. It can solve a few cells though.

The approach is to look at rows and columns in groups of 3. For those values that exist in 2 out of 3 regions, we check if there is only one possible remaining location for that value to go in the $3^{rd}$ region. We can do this because the boxes will rule out some cells that might be otherwise considered. 


In [14]:
puzzle = solve_side_by_side(example, solver.solve_two_out_of_three, show_possibilities=2)

8,9,,4,"{1, 7}","{2, 7}",,5,6
1,4,,3,5,,{7},9,"{2, 7}"
,,,,,,8,"{2, 3}",
9,,,,,,2,,"{1, 7}"
,8,,9,6,5,,4,"{1, 7}"
,"{6, 7}",1,,,,,,5
,,8,"{5, 7}",,,,"{2, 6}",
"{5, 6}",3,,{5},2,1,,7,8
4,2,,,,6,"{9, 5}",1,3
8,9,,4,"{1, 7}","{2, 7}",,5,6
1,4,6,3,5,8,{7},9,2


So how many cells did this method solve?

In [15]:
original.num_empty_cells() - puzzle.num_empty_cells()

11

And that was on the easiest puzzle in the test suite. When solving "manually" it's actually quite a useful and easy technique to apply. My code is pretty awful though and its effectiveness led me to lose motivation for coding any more complicated methods!

## Trying all three

You can see from above that even though the "two out of three" solver only filled in 11 cells it left a few cells that could be solved by "single possibilities" (for example, the lone `{5}` in the bottom right box). The `DeductiveSolver` class can repeatedly call on the different deductive techniques, trying all of them in turn until no effective progress is made. 

For this test we'll move to another puzzle, because we know that the first two methods above can already solve puzzle `#0`. Puzzle `#3` can also be solved, but puzzle `#4` and later can not be solved using the deductive algorithms alone.


In [17]:
example = su.SAMPLE_PUZZLES[3]
solver = su.DeductiveSolver(use_backtracking=False)
puzzle = solve_side_by_side(example, solver.solve)

,2,1,6,,7,8,4,
7,,,,1,,,,3
9,,,,,,,,2
3,,,,,,,,8
2,,,,,,,,7
,9,,,,,,6,
,,4,,,,7,,
,,,2,,1,,,
,,,,8,,,,
5,2,1,6,3,7,8,4,9
7,4,8,9,1,2,6,5,3


In [18]:
example = su.SAMPLE_PUZZLES[4]
solver = su.DeductiveSolver(use_backtracking=False)
puzzle = solve_side_by_side(example, solver.solve, show_possibilities=2)

,,7,5,,"{3, 4}",,"{9, 6}",
1,,"{2, 4}",,,9,8,,
"{2, 5}",6,,"{8, 2}",1,{7},4,3,
8,,5,,,2,"{6, 7}",1,
,,,,,,2,,
,1,,7,,,"{5, 6}",,9
,,3,"{1, 2}",,8,,,4
,4,,9,,,3,,
9,,"{8, 1}",,,6,,2,
,8,7,5,,"{3, 4}",,"{9, 6}",
1,,"{2, 4}",,,9,8,,


So, we very quickly hit a limitation with this approach.

There are some [advanced techniques](https://www.sudokudragon.com/advancedstrategy.htm) but they get more and more complicated and difficult to both code and test. And in many Sudoku guides for players, you'll find a mention of "backtracking":

> When all else fails, there is one technique that is guaranteed to always work, indeed you can solve any Sudoku puzzle using just this one strategy alone. You just work logically through all the possible alternatives in every square in order until you find the allocations that work out. If you choose a wrong option at some stage later you will find a logical inconsistency and have to go back, undoing all allocations and then trying another option
> *[Sudoku Puzzle solving strategies](https://www.sudokudragon.com/sudokustrategy.htm)*

In other words, you can [guess](https://www.sudokudragon.com/sudokuguess.htm). This is called "backtracking" and it's the next strategy I tried.


## Backtracking

In a backtracking solver we're essentially doing a depth-first search, choosing values for empty cells until we find that there are no more valid moves without violating a constraint. At that point, we "back track" -- clearing the "current" cell and going back to our previous guess, choosing a different value this time, then continuing. There's a nice animation to explain the technique on [Wikipedia](https://en.wikipedia.org/wiki/Sudoku_solving_algorithms#Backtracking).

The primary advantage of backtracking is that it can solve any puzzle, given sufficient time. Example `#12` can't be solved by the `DeductiveSolver` but can be solved by `BacktrackingSolver`.

In [26]:
example = su.SAMPLE_PUZZLES[12]
solver = su.BacktrackingSolver()
%time puzzle = solve_side_by_side(example, solver.solve)

,,,7,,,8,,
,,,,4,,,3,
,,,,,9,,,1
6,,,5,,,,,
,1,,,3,,,4,
,,5,,,1,,,7
5,,,2,,,6,,
,3,,,8,,,9,
,,7,,,,,,2
3,2,9,7,1,6,8,5,4
1,7,6,8,4,5,2,3,9


CPU times: user 3.68 s, sys: 16.7 ms, total: 3.69 s
Wall time: 3.74 s


I chose puzzle `#12` because it takes ~3.5 seconds, an unusually long time for most of the solvers here. Most of the other puzzles can be solved much faster (see [Performance Analysis](sudoku/Performance.ipynb)). But this is the weakness of the backtracking algorithm -- it may have to search thousands of "dead ends" beore it stumbles upon the solution.

This particular solver class will count the number of times it "backtracks" and this turned out to be useful when trying to work out [why there were performance variations on the same same puzzle](Sudoku/Constraint%20Propogation%20Variability.ipynb). But we digress...

In [27]:
print(f"Backtracks: {solver.backtrack_count:,}")

Backtracks: 418,147


## Constraint Propogation

A better method, that incorporates backtracking, is known as "constraint propogation." Every time we write in a value for a cell, we can eliminate that value as a possibility in all of that cell's regions (i.e. row, column, and box). If at some point we find that there are now cells with *no possible values* we can terminate the search much earlier than when using backtracking alone.

We can also use knowledge of the constraints to narrow the search path. Instead of going cell-by-cell from the "top left", we can start with cells that have the smallest number of possible values. 

The `ConstraintPropogationSolver` therefore is typically much faster than `BacktrackingSolver` alone, while still being relatively straight forward to code and test. So the puzzle above that took ~3.5 seconds for backtracking to solve takes about ~0.1 seconds when using constraint propogation.

In [28]:
solver = su.ConstraintPropogationSolver()
%time puzzle = solve_side_by_side(example, solver.solve)
print(f"Backtracks: {solver.backtrack_count:,}")

,,,7,,,8,,
,,,,4,,,3,
,,,,,9,,,1
6,,,5,,,,,
,1,,,3,,,4,
,,5,,,1,,,7
5,,,2,,,6,,
,3,,,8,,,9,
,,7,,,,,,2
3,2,9,7,1,6,8,5,4
1,7,6,8,4,5,2,3,9


CPU times: user 140 ms, sys: 5.01 ms, total: 145 ms
Wall time: 197 ms
Backtracks: 2,921


Same puzzle, but only 2K backtracks compared to 418K on the previous attempt.

## Hybrid

Since the deductive methods can't solve all puzzles we have a hybrid approach which will attempt to use the deductive methods as long as possible, and then fall back to backtracking + constraint propogation. It *should* be faster than CP alone, because solving even a few cells with deductive techniques reduces the search space for any kind of search-based approach.

The `DeductiveSolver` is in fact a child class of `ConstraintPropogationSolver`, and once its deductive algorithms stop yielding solved cells, it calls the parent class `solve` method to complete the puzzle using constraint propogation and backtracking. 

This approach can dramatically speed up the performance on some puzzles. For example, puzzle `#16` typically takes ~3.5 seconds to solve with just constraint propogation, but if we run it through the deductive solver first, the time is around ~0.5 seconds, because it's able to fill in 6 cells before falling back to backtracking.


In [29]:
example = su.SAMPLE_PUZZLES[16]
puzzle = su.SudokuPuzzle(starting_grid=su.from_string(example['puzzle']))
solver = su.ConstraintPropogationSolver()
%time solver.solve(puzzle)
print(f"Backtracks: {solver.backtrack_count:,}")

CPU times: user 3.24 s, sys: 13.1 ms, total: 3.25 s
Wall time: 3.36 s
Backtracks: 138,898


In [31]:
solver = su.DeductiveSolver(use_backtracking=True)
%time puzzle = solve_side_by_side(example, solver.solve)
print(f"Backtracks: {solver.backtrack_count:,}")

6,,2,,5,,,,
,,,,,4,,3,
,,,,,,,,
4,3,,,,8,,,
,1,,,,,2,,
,,,,,,7,,
5,,,2,7,,,,
,,,,,,,8,1
,,,6,,,,,
6,8,2,1,5,3,4,7,9
9,5,1,7,6,4,8,3,2


CPU times: user 542 ms, sys: 6.61 ms, total: 549 ms
Wall time: 615 ms
Backtracks: 19,179


## SAT Solver

At this point I got curious about the other ways people have found to solve these puzzles. I came across [Ilan Schnell's SAT approach](http://ilan.schnell-web.net/prog/sudoku/) (code [here](https://github.com/ContinuumIO/pycosat/blob/master/examples/sudoku.py)). By chance I had been reading about [making Conda's package installation performance faster](https://www.anaconda.com/blog/understanding-and-improving-condas-performance) and read that part of the process for reconciling package dependencies included expressing the package constraints as an SAT problem. I took it as a sign.

There's a reasonably clear [introduction to SAT solvers](https://codingnest.com/modern-sat-solvers-fast-neat-underused-part-1-of-n/) which includes how they can be applied to Sudoku puzzles. The implementation in [SATSolver](puzzle/sudoku.py) however is Schnell's work, and uses [pycosat](https://pypi.org/project/pycosat/), a Python wrapper for [PicoSAT](http://fmv.jku.at/picosat/), for the actual SAT solving part. This library is very fast, thanks to a combination of optimised C code and about 20 years of academic research. In tests, it never took longer than 50 ms to solve any puzzle.


In [33]:
solver = su.SATSolver()
%time puzzle = solve_side_by_side(example, solver.solve)

6,,2,,5,,,,
,,,,,4,,3,
,,,,,,,,
4,3,,,,8,,,
,1,,,,,2,,
,,,,,,7,,
5,,,2,7,,,,
,,,,,,,8,1
,,,6,,,,,
6,8,2,1,5,3,4,7,9
9,5,1,7,6,4,8,3,2


CPU times: user 38.8 ms, sys: 2.62 ms, total: 41.4 ms
Wall time: 53.9 ms


## Other Algorithms

There are a few other ways I came across but didn't implement -- the SAT solver was just too fast. By this point I'd also found Peter Norvig's [Solving Every Sudoku Puzzle](https://norvig.com/sudoku.html), which is using constraint propogation, is written in pure Python, and is pretty damn fast compared to my implementation. But he's a genius and comparing yourself to a genius is a pretty fast route to total demotivation so I stopped. :-)

For what it's worth the other approaches that might be worth looking at are:

1. [Algorithm X](https://www.cs.mcgill.ca/~aassaf9/python/algorithm_x.html), or "[Dancing Links](https://www.kevinhooke.com/2019/01/22/revisiting-donald-knuths-algorithm-x-and-dancing-links-to-solve-sudoku-puzzles/)"
2. [Mixed Integer Programming](http://yetanothermathprogrammingconsultant.blogspot.com/2016/10/mip-modeling-from-sudoku-to-kenken.html) (MIP), although I need to understand [Linear Programming](https://towardsdatascience.com/using-integer-linear-programming-to-solve-sudoku-puzzles-15e9d2a70baa) first (or use [PuLP](https://www.coin-or.org/PuLP/CaseStudies/a_sudoku_problem.html)).
3. Finally, I'm not confident I've 100% understood SAT solvers, and there [might be some optimizations](https://gist.github.com/nickponline/9c91fe65fef5b58ae1b0) that could make it even faster.


---
# Where to from here?

0. Go back to the [table of contents](#contents) (in this notebook).
1. [Benchmarking the performance](Sudoku/Performance.ipynb) of the different algorithms.
2. [Checking for cheating](Sudoku/Cheating.ipynb).
3. Investigating [variations in the performance of constraint propogation](Sudoku/Constraint%20Propogation%20Variability.ipynb).
4. Testing against [larger Sudoku puzzle sizes](Sudoku/Larger%20Puzzles.ipynb) (up to 25 x 25 cells).

See also [Sudoku TODOs](Sudoku/TODO.ipynb); [Sources](#sources); or [Lessons Learned](#lessons).

---
# Sources <a class="anchor" id="sources"></a>

Part of this exercise was to learn Python and Jupyter skills while also solving a problem that I found interesting. So at first I avoided reading other people's solutions to solving Sudoku. Eventually though I got curious and found the below sources useful.

* [Solving Every Sudoku Puzzle](https://norvig.com/sudoku.html), by Peter Norvig. The "[Top 95](data/sudoku_9x9/top95.txt)" and "[Hardest](data/sudoku_9x9/hardest.txt)" puzzle examples in the data directory come from there.
* [Sudoku solving algorithms](https://en.wikipedia.org/wiki/Sudoku_solving_algorithms) on Wikipedia, which links to some sample puzzles (on Flickr of all places).
* [AI Sudoku](http://www.aisudoku.com/index_en.html) -- collection of really hard puzzles.
* The [sudoku.py](puzzle/sudoku.py) class has URLs to where I found some of the sample puzzles. I've attempted to use labels for them that credit the source, although it's not always clear where the original puzzle came from.
* Also used examples from [Simple sudoku solver using constraint propagation](https://gpicavet.github.io/jekyll/update/2017/12/16/sudoku-solver.html) (Grégory Picavet's Blog).
* Test file [sudoku17.txt](data/sudoku_9x9/sudoku17.txt) was found via  [Prof. Gordon Royle's article on The Conversation](https://theconversation.com/good-at-sudoku-heres-some-youll-never-complete-5234).
* The [Sudoku Dragon](https://www.sudokudragon.com/) site has descriptions of the "deductive" algorithms I used, and was also a source of some of the test puzzles, including so called "[unsolvable](https://www.sudokudragon.com/unsolvable.htm)" puzzles.


---

# Lessons Learned <a class="anchor" id="lessons"></a>

## Jupyter Tips

### Split Notebooks

The original versions of this notebook got quite long and it became difficult to keep it all clear and make it readable. Splitting the notebooks and using links in the Markdown sections makes it easier to maintain (and I hope, read).

### Use Modules

Exploratory code in the notebook is fine but very quickly you want to move code into a Python module. I wish there was some way to link to specific lines (or better yet, classes and methods) but for now you can at least link to the source files.

### Formatting Tables

By default all tables are formatted the same way and this is optimised for Pandas DataFrames. You can insert additional CSS using `display(HTML(...))` and if you give the table its own CSS `class` then you can have different table looks without them inerfering with each other.

### Maths

Complex expressions can be built with Latex-like markup, see [Mixing Markdown and Tex](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Typesetting%20Equations.html).

Simple math expressions can be done inline with \\$ to begin and end an expression in Markdown. For example, to print $\sqrt{9}=3$:

    $\sqrt{9}=3$
